In [ ]:
import csv
from io import StringIO
import json
from os import environ
from os.path import isfile, join
import pathlib
from subprocess import Popen

from PIL import Image
import requests

from utils.text import to_slug
from utils.image import resize_img, crop_face, calculate_dominant_color

IMAGES_DIR = join('.', 'imgs')
IMAGES_DIR_RAW = join(IMAGES_DIR, '00_raw')
IMAGES_DIR_HD = join(IMAGES_DIR, '01_hd')
IMAGES_DIR_THUMB = join(IMAGES_DIR, '02_thumb')
IMAGES_DIR_FACES = join(IMAGES_DIR, '00_raw_faces')

WEB_DIR = join('..', 'cordiais-web', 'public')
WEB_DIR_IMAGES = join(WEB_DIR, 'imgs', 'obras')
WEB_DIR_FACES = join(WEB_DIR, 'imgs', 'faces')
WEB_DIR_DATA = join(WEB_DIR, 'data')
WEB_DATA_FILE = join(WEB_DIR_DATA, 'obras.json')

FACE_API_URL = 'https://api-us.faceplusplus.com/facepp/v3/detect'
FACE_API_DATA = {
    'api_key': environ.get('FACEPP_KEY'),
    'api_secret': environ.get('FACEPP_SECRET'),
    'return_attributes': 'emotion,gender,age,facequality,beauty'
}

In [ ]:
with open(WEB_DATA_FILE, 'r') as in_json:
    obras_json = json.load(in_json)

In [ ]:
counts = []
for i in range(10):
    counts.append([])

for slug, v in obras_json.items():
    fcnt = v['faces']
    counts[v['faces']].append(slug)

list(map(len, counts))

In [ ]:
slugs_multi = [s for l in counts[2:] for s in l]
len(slugs_multi)

In [ ]:
def process_file(img_file):
    files = {
        'image_file': open(img_file, 'rb')
    }

    res = requests.post(FACE_API_URL, files=files, data=FACE_API_DATA)
    res_content = json.loads(res.text)
    res_content["ok"] = res.ok

    return res_content

In [ ]:
def display_faces(obra_slug, type="HD"):
    if type == 'WEB':
        img_file = join(WEB_DIR_IMAGES, '%s_%s.jpg' % (obra_slug, 'web'))
    elif type == 'THUMB':
        img_file = join(IMAGES_DIR_THUMB, '%s_%s.jpg' % (obra_slug, 'thumb'))
    else:
        img_file = join(IMAGES_DIR_HD, '%s_%s.jpg' % (obra_slug, 'hd'))

    res = process_file(img_file)
    img = Image.open(img_file).convert('RGB')

    if res["ok"]:
        if res['face_num'] > 0:
            by_female = sorted(
                [f for f in res['faces'][:5] if f['attributes']['gender']['value'] == 'Female'],
                key=lambda x: x['face_rectangle']['width'],
                reverse=True)
            by_size = sorted(
                [f for f in res['faces'][:5]],
                key=lambda x: x['face_rectangle']['width'],
                reverse=True)
            top_rect = by_female[0]['face_rectangle'] if len(by_female) > 0 else by_size[0]['face_rectangle']
            gen_id = "F" if len(by_female) > 0 else "?"
            top_image = img.crop((
                top_rect['left'],
                top_rect['top'],
                top_rect['left']+top_rect['width'],
                top_rect['top']+top_rect['height']))
            print("%s: " % gen_id, obra_slug)
            display(top_image)
    else:
        print('get_face_attributes ERROR: %s' % json.dumps(res, sort_keys=True, indent=2))

In [ ]:
for obra_slug in slugs_multi:
    display_faces(obra_slug)

In [ ]:
linda_slug = 'aladino-divani_retrato-de-linda-divani'
linda_file = join(IMAGES_DIR_HD, '%s_%s.jpg' % (linda_slug, 'hd'))
process_file(linda_file)